In [2]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.4 MB/s eta 0:00:00


In [3]:
from sentence_transformers import SentenceTransformer, losses
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm_notebook
from IPython.display import clear_output
from torch.optim.lr_scheduler import StepLR
import copy
import time


seed = 0
torch.manual_seed(seed)
np.random.seed(seed)

base_dir = '/content/drive/MyDrive/SemEval/'
train_data_dir = base_dir + 'train_data.csv'
val_data_dir = base_dir + 'val-60.csv'

In [4]:
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
class TextTrainDataset(Dataset):
  def __init__(self, df):
    self.df = df

  def __len__(self):
    return len(self.df)

  def __getitem__(self, idx):
    question = self.df.iloc[idx]['question']
    answer = self.df.iloc[idx]['answer']
    distractor = self.df.iloc[idx]['distractor']
    return question, answer, distractor


class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.embedding = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', ).to(device=device)
    self.linear1 = nn.Linear(768, 64, device=device)
    self.linear2 = nn.Linear(768*2, 64, device=device)
    self.tanh = nn.Tanh()

  def embed(self, question, answer, distractor):
    question = torch.from_numpy(self.embedding.encode(question)).to(device)
    answer = torch.from_numpy(self.embedding.encode(answer)).to(device)
    distractor = torch.from_numpy(self.embedding.encode(distractor)).to(device)
    return question, answer, distractor

  def forward(self, question, answer, distractor):
    question = question.to(device)
    answer = answer.to(device)
    distractor = distractor.to(device)

    anchor = question
    positive = torch.cat((question, answer), dim=1)
    negative = torch.cat((question, distractor), dim=1)

    anchor = self.tanh(self.linear1(anchor).to(device))
    positive = self.tanh(self.linear2(positive).to(device))
    negative = self.tanh(self.linear2(negative).to(device))

    return anchor, positive, negative

In [7]:
def make_dataset(df):
  new_df = pd.DataFrame(columns=['question', 'answer', 'distractor'])
  for i, row in df.iterrows():
    q = row['question']
    ans = row['answer']
    distractor1 = row['distractor1']
    distractor2 = row['distractor2']
    temp = pd.DataFrame({
        'question': [q, q],
        'answer': [ans, ans],
        'distractor': [distractor1, distractor2]
    })
    new_df = pd.concat([new_df, temp], ignore_index=True)

  return TextTrainDataset(new_df)

In [8]:
def seperate_kfold(main_df, k, val_size):
  main_df = main_df.sample(frac=1, random_state=seed).reset_index(drop=True)
  dfs = []
  interval_len = main_df.shape[0] // k
  assert val_size < interval_len
  for i in range(k):
    start = np.random.randint(i * interval_len, (i+1) * interval_len - val_size)
    val_df = main_df.loc[start:start + val_size]
    train_df = main_df.drop(main_df.index[start:start+val_size]).reset_index(drop=True)
    dfs.append((train_df, val_df))
  return dfs

In [9]:
def similarity(q, p, n):
  # return torch.dist(q, p, p=2) - torch.dist(q, n, p=2)
  cos = nn.CosineSimilarity(dim=1, eps=1e-6)
  d1 = cos(q, p)
  d2 = cos(q, n)
  return d1 - d2

def inference(model, question, option1, option2, option3):
  model.eval()
  samples = [model.embed(question, option1, option2), model.embed(question, option2, option1),
             model.embed(question, option1, option3), model.embed(question, option3, option1),
             model.embed(question, option2, option3), model.embed(question, option3, option2),
             ]
  options = [option1, option2, option1, option3, option2, option3]
  answers = []
  for i, s in enumerate(samples):
    q, ans, dis = model(torch.reshape(s[0], (1,) + s[0].shape), torch.reshape(s[1], (1,) + s[1].shape), torch.reshape(s[2], (1,) + s[2].shape))
    answers.append((i, similarity(q, ans, dis)))
  sorted_options = sorted(answers, key=lambda x:x[1])
  best_ans = sorted_options[-1][0]
  return options[best_ans]


def accuracy(model, df):
  total = 0
  correct = 0
  for i, row in df.iterrows():
    q = row['question']
    ans = row['answer']
    dis1 = row['distractor1']
    dis2 = row['distractor2']
    options = [ans, dis1, dis2]
    random.shuffle(options)
    pred = inference(model, q, options[0], options[1], options[2])
    if pred == ans:
      correct += 1
    total += 1
  return correct / total * 100

In [10]:
class CosineTripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(CosineTripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6)
        positive_sim = cos_sim(anchor, positive)
        negative_sim = cos_sim(anchor, negative)
        # losses = nn.functional.relu(negative_sim - positive_sim + self.margin)
        losses = negative_sim - positive_sim + self.margin
        return losses.mean()


def train_model(model, loader, val_df, n_epochs):
  # loss_fn = nn.TripletMarginLoss(margin=1.0, p=2)
  loss_fn = CosineTripletLoss(2)
  optimizer = optim.Adam(model.parameters())
  # scheduler = StepLR(optimizer, step_size=2, gamma=0.5)
  model = model.to(device)
  best_model = copy.deepcopy(model)
  best_val_acc = -1000
  train_loss = 10000
  for epoch in tqdm_notebook(range(n_epochs)):
    model.train()
    total_loss = 0.0
    for i, batch in enumerate(loader):
      question, answer, distractor = batch
      question, answer, distractor = model.embed(question, answer, distractor)

      anchor, positive, negative = model(question, answer, distractor)

      loss = loss_fn(anchor, positive, negative)
      total_loss += loss.item() / len(batch)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    val_acc = accuracy(model, val_df)
    if val_acc > best_val_acc:
      best_val_acc = val_acc
      train_loss = total_loss
      best_model = copy.deepcopy(model)
    elif val_acc == best_val_acc and total_loss < train_loss:
      best_val_acc = val_acc
      train_loss = total_loss
      best_model = copy.deepcopy(model)

    print(f'Epoch: {epoch+1}, Loss: {total_loss}, Val_Acc: {val_acc}')

  return best_model, best_val_acc, train_loss

In [11]:
def train_k_models(dfs, k, n_epochs, batch_size=16):
  results = []
  for i in range(k):
    print(f'model {i}')
    model = Model()
    train_df, val_df = dfs[i]
    train_dataset = make_dataset(train_df)
    val_dataset = make_dataset(val_df)
    loader = DataLoader(train_dataset, batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size)
    best_model, best_val_acc, train_loss = train_model(model, loader, val_df, n_epochs)
    results.append({
        'model': best_model,
        'name': f'model_{i}',
        'val_acc': best_val_acc,
        'train_loss': train_loss
    })
    clear_output(wait=True)
  return results

In [12]:
def pick_best_model(results):
  res = sorted(results, key=lambda x:x['val_acc'] / x['train_loss'])
  for r in res:
    r['key'] = r['val_acc'] / r['train_loss']
  print(res)
  best_model = res[(len(results) - 1) // 2]
  return best_model

In [13]:
main_df = pd.read_csv(train_data_dir)[['question', 'answer', 'distractor1', 'distractor2']]

In [14]:
k = 3
val_size = 20
n_epochs = 10
batch_size = 16
dfs = seperate_kfold(main_df, k, val_size)

In [15]:
results = train_k_models(dfs, k, n_epochs, batch_size)
results

[{'model': Model(
    (embedding): SentenceTransformer(
      (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
      (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
      (2): Normalize()
    )
    (linear1): Linear(in_features=768, out_features=64, bias=True)
    (linear2): Linear(in_features=1536, out_features=64, bias=True)
    (tanh): Tanh()
  ),
  'name': 'model_0',
  'val_acc': 95.23809523809523,
  'train_loss': 4.074779701729616},
 {'model': Model(
    (embedding): SentenceTransformer(
      (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
      (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 

In [16]:
def convert_valdf(df):
  new_df = pd.DataFrame(columns=['question', 'answer', 'distractor1', 'distractor2'])
  for i, row in df.iterrows():
    if row['option1'] == row['answer']:
      values = [row['question'], row['answer'], row['option2'], row['option3']]
    elif row['option2'] == row['answer']:
      values = [row['question'], row['answer'], row['option1'], row['option3']]
    elif row['option3'] == row['answer']:
      values = [row['question'], row['answer'], row['option1'], row['option2']]
    new_df.loc[len(new_df)] = values
  return new_df

In [17]:
best_model = pick_best_model(results)

main_val_df = convert_valdf(pd.read_csv(val_data_dir))

for model in results:
  print(model['name'], ': ', accuracy(model['model'], main_val_df))
accuracy(best_model['model'], main_val_df)

[{'model': Model(
  (embedding): SentenceTransformer(
    (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
    (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
    (2): Normalize()
  )
  (linear1): Linear(in_features=768, out_features=64, bias=True)
  (linear2): Linear(in_features=1536, out_features=64, bias=True)
  (tanh): Tanh()
), 'name': 'model_0', 'val_acc': 95.23809523809523, 'train_loss': 4.074779701729616, 'key': 23.372575257913855}, {'model': Model(
  (embedding): SentenceTransformer(
    (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
    (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'poo

75.0

In [18]:
test_dir = base_dir + 'new_test_data_nolabel.csv'
test_df = pd.read_csv(test_dir)

def predict(model, df):
  for i, row in df.iterrows():
    question = row['question']
    option1 = row['option1']
    option2 = row['option2']
    option3 = row['option3']
    prediction = inference(model, question, option1, option2, option3)
    if option1 == prediction:
      id = 0
    elif option2 == prediction:
      id = 1
    elif option3 == prediction:
      id = 2
    df.loc[i, 'pred_id'] = int(id)

  return df

def write_answer_id(df, path):
  with open(path, 'w') as f:
    for i, row in df.iterrows():
      f.write(str(int(row['pred_id']))+'\n')

In [20]:
test_df = predict(best_model['model'], test_df)
test_df

,Unnamed: 0,question,option1,option2,option3,option4,pred_id
0,0,"In a small village, two farmers are working in...",The lazy farmer is his mother.,The lazy farmer is not a responsible father as...,The diligent farmer devoted himself to the far...,None of above.,0.0
1,1,Romeo and Juliet are discovered dead on the be...,They were sleeping and scared by the sound of ...,The rumble of the train moved the shelf which ...,Romeo and Juliet are fish. The rumble of the t...,None of above.,2.0
2,2,How many years in your life it happens that be...,In ech leap year.,In the first year of graduation.,It happens every year.,None of above.,0.0
3,3,Who would serve as the team's captain if a cru...,The first officer.,The captain.,The second officer.,None of above.,1.0
4,4,"In one city, 5% of the population has an unlis...",One hundred people.,Ninty-five people.,Five people.,None of above.,0.0
...,...,...,...,...,...,...,...
115,115,A professional fisherman caught 30 fish during...,Two.,One.,Three.,None of above.,1.0
116,116,Bob was working on a project when suddenly int...,He needed a membership to search in google.,He searched the google in a wrong way.,The internet was disconnected so he couldn't s...,None of above.,2.0
117,117,"He has wed numerous women, but never himself. ...",A teacher.,A preacher.,A laywer.,None of above.,0.0
118,118,You walk into a room and see a bed and lie on ...,"Thirty-six, as there are eighteen animals.","Six. The bed's four legs, plus your two legs.","Seventy-two, as there are eighteen animals.",None of above.,0.0


In [21]:
path = base_dir + 'answer_sen.txt'
write_answer_id(test_df, path)